In [19]:
# Notebook: 02 - Data Transform
# Objetivo: ler o JSON gerado em notebooks/01, inspecionar e aplicar transformacoes de tipos e nomes
import os
import sys
from pathlib import Path
import pandas as pd
import json
import re
from dotenv import load_dotenv


# Garante que a raiz do projeto esteja no sys.path (mesma abordagem do notebook 01)
project_root = Path(os.getcwd()).resolve().parents[0]
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from src.normalizers.col_normalizer import normalize_columns

# Carrega variaveis de ambiente, se houver .env
load_dotenv()

False

In [20]:
# Função utilitária para padronizar nomes de colunas para snake_case
# Remove acentos, caracteres não alfanuméricos, converte CamelCase -> snake_case e normaliza underscores
import unicodedata

def _clean_col_name(name: str) -> str:
    """Limpa um único nome de coluna e retorna em snake_case.

    Regras:
    - Normaliza unicode (remove acentos)
    - Substitui espaços e caracteres não alfanuméricos por underscore
    - Converte CamelCase para snake_case
    - Remove underscores duplicados e trims de '_' nas extremidades
    """
    if not isinstance(name, str):
        return name
    # Normaliza e remove acentos
    s = unicodedata.normalize('NFKD', name)
    s = ''.join(ch for ch in s if not unicodedata.combining(ch))
    # Remove caracteres ordinais comuns
    s = s.replace('º', '').replace('ª', '')
    # Substitui espaços por underscore
    s = re.sub(r"\s+", '_', s)
    # Substitui qualquer caractere que não seja letra, número ou underscore por underscore
    s = re.sub(r'[^0-9A-Za-z_]', '_', s)
    # CamelCase -> snake_case
    s = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', s)
    s = re.sub('([a-z0-9])([A-Z])', r'\1_\2', s)
    s = s.lower()
    # Collapse underscores
    s = re.sub(r'_+', '_', s).strip('_')
    return s


def normalize_columns(df: 'pd.DataFrame', inplace: bool = False) -> 'pd.DataFrame':
    """Padroniza os nomes das colunas de um DataFrame para snake_case.

    Parâmetros:
    - df: DataFrame alvo
    - inplace: se True altera o DataFrame passado e retorna ele; caso contrário retorna uma cópia renomeada

    Retorna:
    - DataFrame com colunas renomeadas
    """
    mapping = {c: _clean_col_name(c) for c in df.columns}
    if inplace:
        df.rename(columns=mapping, inplace=True)
        return df
    else:
        return df.rename(columns=mapping)

# Exemplo de uso (apenas para referência):
# df = normalize_columns(df)
# df_projetos = normalize_columns(df_projetos, inplace=True)


# 02_data_transform.ipynb

Este notebook carrega o arquivo JSON gerado por `01_data_extraction.ipynb`, realiza inspeção rápida das colunas e aplica transformações de tipo e padronização dos nomes das colunas (snake_case). Ao final, salva uma versão processada em `data/processed/` nos formatos parquet e json para usos posteriores.

Boas práticas aplicadas:
- Leitura tolerante a objetos vazios
- Padronização de nomes de colunas (snake_case)
- Conversão segura de datas e booleans
- Remoção de colunas muito esparsas (muitos NAs)
- Documentação via células Markdown explicativas

In [21]:
# Paths de entrada/saida
ROOT = project_root
RAW_PATH = ROOT / 'data' / 'raw' / 'dados_obras_gov.json'
PROCESSED_DIR = ROOT / 'data' / 'processed'
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

print('Raw path:', RAW_PATH)
print('Processed dir:', PROCESSED_DIR)

# Leitura segura do JSON: se o arquivo contem objetos vazios, json.load pode falhar em alguns casos, entao usamos pandas.read_json com orientacao por records
try:
    df = pd.read_json(RAW_PATH, orient='records')
except ValueError:
    # fallback: carrega como texto e ignora objetos invalidos
    with open(RAW_PATH, 'r', encoding='utf-8') as f:
        raw_text = f.read()
    data = json.loads(raw_text)
    df = pd.json_normalize(data)

print('Dados carregados: rows=', len(df))
df.head(3)


Raw path: E:\VS-Code\Lappis-PS\data\raw\dados_obras_gov.json
Processed dir: E:\VS-Code\Lappis-PS\data\processed
Dados carregados: rows= 800


,idUnico,nome,cep,endereco,descricao,funcaoSocial,metaGlobal,dataInicialPrevista,dataFinalPrevista,dataInicialEfetiva,...,observacoesPertinentes,isModeladaPorBim,dataSituacao,tomadores,executores,repassadores,eixos,tipos,subTipos,fontesDeRecurso
0,50379.53-54,DL - 304/2024 - Contratação de instituição par...,None,None,Contratação de instituição para execução de se...,Ampliação da capacidade de trafego visando a m...,Projetos Básicos e Executivos de Engenharia,2024-12-20,2027-12-05,None,...,None,0.0,2024-12-20,[],[{'nome': 'DEPARTAMENTO NACIONAL DE INFRAESTRU...,[],"[{'id': 3, 'descricao': 'Econômico'}]","[{'id': 25, 'descricao': 'Rodovia', 'idEixo': 3}]","[{'id': 4, 'descricao': 'Acessos Terrestres', ...","[{'origem': 'Federal', 'valorInvestimentoPrevi..."
1,42724.53-27,Escola Classe Crixá São Sebastião,None,None,"Construção de Escola em Tempo Integral, Escola...",A construção da nova escola beneficiará 977 es...,"Construção de Escola em Tempo Integral, Escola...",2024-09-02,2028-09-02,None,...,None,0.0,2025-09-05,[],[{'nome': 'SECRETARIA DE ESTADO DE EDUCACAO DO...,[{'nome': 'FUNDO NACIONAL DE DESENVOLVIMENTO D...,"[{'id': 4, 'descricao': 'Social'}]","[{'id': 46, 'descricao': 'Educação', 'idEixo':...","[{'id': 84, 'descricao': 'Educação', 'idTipo':...","[{'origem': 'Federal', 'valorInvestimentoPrevi..."
2,19970.53-78,Reajuste do Contrato 45/2021 - Contrução do Ce...,70.602-600,"SAIS Área Especial 3, Setor Policial Sul",Reajuste do Contrato 45/2021 - Construção do C...,Contribuir para a melhor formação dos bombeiro...,Construção de um novo centro de formação e de ...,2021-09-14,2024-08-28,None,...,None,0.0,2023-02-06,[],[{'nome': 'CORPO DE BOMBEIROS MILITAR DO DISTR...,[{'nome': 'CORPO DE BOMBEIROS MILITAR DO DISTR...,"[{'id': 1, 'descricao': 'Administrativo'}]","[{'id': 1, 'descricao': 'Segurança Pública', '...","[{'id': 59, 'descricao': 'Obras em Imóveis de ...","[{'origem': 'Federal', 'valorInvestimentoPrevi..."


## 1) Inspeção inicial
A seguir mostramos informações básicas e as colunas detectadas. Isso ajuda a decidir quais transformações aplicar (datas, booleans, colunas aninhadas etc.).

In [22]:
# Informacoes basicas
print('shape:', df.shape)
print('Sample dtypes:')
print(df.dtypes.head(30))

# Percentual de valores nulos por coluna (ordenado)
na_frac = df.isna().mean().sort_values(ascending=False)
na_frac.head(30)

# Lista de colunas
list(df.columns)[:80]

shape: (800, 31)
Sample dtypes:
idUnico                                object
nome                                   object
cep                                    object
endereco                               object
descricao                              object
funcaoSocial                           object
metaGlobal                             object
dataInicialPrevista                    object
dataFinalPrevista                      object
dataInicialEfetiva                     object
dataFinalEfetiva                       object
dataCadastro                           object
especie                                object
natureza                               object
naturezaOutras                         object
situacao                               object
descPlanoNacionalPoliticaVinculado     object
uf                                     object
qdtEmpregosGerados                     object
descPopulacaoBeneficiada               object
populacaoBeneficiada                   object
ob

['idUnico',
 'nome',
 'cep',
 'endereco',
 'descricao',
 'funcaoSocial',
 'metaGlobal',
 'dataInicialPrevista',
 'dataFinalPrevista',
 'dataInicialEfetiva',
 'dataFinalEfetiva',
 'dataCadastro',
 'especie',
 'natureza',
 'naturezaOutras',
 'situacao',
 'descPlanoNacionalPoliticaVinculado',
 'uf',
 'qdtEmpregosGerados',
 'descPopulacaoBeneficiada',
 'populacaoBeneficiada',
 'observacoesPertinentes',
 'isModeladaPorBim',
 'dataSituacao',
 'tomadores',
 'executores',
 'repassadores',
 'eixos',
 'tipos',
 'subTipos',
 'fontesDeRecurso']

In [23]:
print(df["situacao"])

0      Cadastrada
1       Cancelada
2      Cadastrada
3      Cadastrada
4      Cadastrada
          ...    
795    Cadastrada
796    Cadastrada
797     Cancelada
798    Cadastrada
799    Cadastrada
Name: situacao, Length: 800, dtype: object


## 3) Conversão de tipos
Identificamos colunas que devem ser convertidas para datas e booleans. Usa-se um mapeamento baseado em nomes de coluna comuns detectados no dataset.

In [24]:
# Colunas de datas
date_columns = [
    'dataInicialPrevista', 'dataFinalPrevista', 'dataInicialEfetiva',
    'dataFinalEfetiva', 'dataCadastro', 'dataSituacao'
]
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce') # 'coerce' transforma erros em NaT (Not a Time)

# Colunas numéricas
numeric_columns = ['qdtEmpregosGerados', 'populacaoBeneficiada']
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce') # 'coerce' transforma erros em NaN

# Coluna booleana
# O tipo 'boolean' (com B maiúsculo) suporta valores nulos (NA)
df['isModeladaPorBim'] = df['isModeladaPorBim'].astype('boolean')

text_columns = [
    'idUnico', 'nome', 'cep', 'endereco', 'descricao', 'funcaoSocial',
    'metaGlobal', 'especie', 'natureza', 'naturezaOutras', 'situacao',
    'descPlanoNacionalPoliticaVinculado', 'uf', 'descPopulacaoBeneficiada',
    'observacoesPertinentes'
]

for col in text_columns:
    # Verifique se a coluna existe antes de converter
    if col in df.columns:
        df[col] = df[col].astype('string') # A conversão acontece aqui!


print("\nTipos de Dados Corrigidos (DataFrame Principal):")
print(df.info())


Tipos de Dados Corrigidos (DataFrame Principal):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 31 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   idUnico                             800 non-null    string        
 1   nome                                800 non-null    string        
 2   cep                                 362 non-null    string        
 3   endereco                            403 non-null    string        
 4   descricao                           800 non-null    string        
 5   funcaoSocial                        800 non-null    string        
 6   metaGlobal                          800 non-null    string        
 7   dataInicialPrevista                 798 non-null    datetime64[ns]
 8   dataFinalPrevista                   798 non-null    datetime64[ns]
 9   dataInicialEfetiva                  27 non-null 

In [25]:
# Tabela principal (projetos) - vamos manter apenas as colunas "simples"
colunas_aninhadas = ['tomadores', 'executores', 'repassadores', 'eixos', 'tipos', 'subTipos', 'fontesDeRecurso']
df_projetos = df.drop(columns=colunas_aninhadas)

# Função auxiliar para evitar repetição de código
def normalizar_coluna(df, coluna_id, coluna_aninhada):
    # Filtra linhas onde a coluna aninhada não está vazia
    df_filtrado = df[[coluna_id, coluna_aninhada]].dropna(subset=[coluna_aninhada]).explode(coluna_aninhada)
    
    # Se após o explode a coluna estiver vazia, retorna um DataFrame vazio
    if df_filtrado.empty:
        return pd.DataFrame()

    # Normaliza o dicionário em colunas
    df_normalizado = pd.json_normalize(df_filtrado[coluna_aninhada])
    
    # Adiciona o ID do projeto para o join
    df_final = df_normalizado.set_index(df_filtrado.index).join(df_filtrado[coluna_id])
    
    # Reorganiza as colunas para ter o ID primeiro
    return df_final[[coluna_id] + [col for col in df_final.columns if col != coluna_id]]


# Criando os DataFrames "filhos" (ignorei tomadores, muito semelhante ao executadores)
df_executores = normalizar_coluna(df, 'idUnico', 'executores')
df_eixos = normalizar_coluna(df, 'idUnico', 'eixos')
df_tipos = normalizar_coluna(df, 'idUnico', 'tipos')
df_subtipos = normalizar_coluna(df, 'idUnico', 'subTipos')
df_fontes_recurso = normalizar_coluna(df, 'idUnico', 'fontesDeRecurso')

# Transformando em string as colunas necessarias de cada DataFrame filho
df_executores["nome"] = df_executores["nome"].astype("string")
df_eixos["descricao"] = df_eixos["descricao"].astype("string")
df_tipos["descricao"] = df_tipos["descricao"].astype("string")
df_subtipos["descricao"] = df_subtipos["descricao"].astype("string")
df_fontes_recurso["origem"] = df_fontes_recurso["origem"].astype("string")

print("\n--- DataFrames Normalizados ---")
print("\nDataFrame de Projetos (Principal):")
print(df_projetos.head())

print("\nDataFrame de Executores:")
print(df_executores.head())

print("\nDataFrame de Fontes de Recurso:")
print(df_fontes_recurso.head())


--- DataFrames Normalizados ---

DataFrame de Projetos (Principal):
       idUnico                                               nome         cep  \
0  50379.53-54  DL - 304/2024 - Contratação de instituição par...        <NA>   
1  42724.53-27                  Escola Classe Crixá São Sebastião        <NA>   
2  19970.53-78  Reajuste do Contrato 45/2021 - Contrução do Ce...  70.602-600   
3  24797.53-15  Implantação de Passarelas nas Estradas Parque ...        <NA>   
4  24822.53-70  obra de construção da  Cabine de Medição, loca...        <NA>   

                                   endereco  \
0                                      <NA>   
1                                      <NA>   
2  SAIS Área Especial 3, Setor Policial Sul   
3                                      <NA>   
4                                      <NA>   

                                           descricao  \
0  Contratação de instituição para execução de se...   
1  Construção de Escola em Tempo Integral, Escola

In [27]:
# Normalizando o titula das colunas
for df_i in [df_projetos, df_executores, df_eixos, df_tipos, df_subtipos, df_fontes_recurso]:
    df_i = normalize_columns(df_i, True)

In [31]:
# Remover espaços em branco de colunas de texto (categóricas)

text_columns = ['especie', 'natureza', 'situacao', 'uf']

for col in text_columns:
    if col in df_projetos.columns:
        # Cria a máscara para identificar valores que não são nulos
        mask = df_projetos[col].notna()
        
        # Usa .loc para aplicar a transformação APENAS nas células onde a máscara é True
        df_projetos.loc[mask, col] = df_projetos.loc[mask, col].astype(str).str.strip().str.upper()

# Estratégia para valores nulos (exemplo)
# Para 'qdtEmpregosGerados', talvez 0 seja um preenchimento razoável.
df_projetos['qdt_empregos_gerados'] = df_projetos['qdt_empregos_gerados'].fillna(0)

print("\nDataFrame de Projetos Após Limpeza:")
print(df_projetos.head())


DataFrame de Projetos Após Limpeza:
      id_unico                                               nome         cep  \
0  50379.53-54  DL - 304/2024 - Contratação de instituição par...        <NA>   
1  42724.53-27                  Escola Classe Crixá São Sebastião        <NA>   
2  19970.53-78  Reajuste do Contrato 45/2021 - Contrução do Ce...  70.602-600   
3  24797.53-15  Implantação de Passarelas nas Estradas Parque ...        <NA>   
4  24822.53-70  obra de construção da  Cabine de Medição, loca...        <NA>   

                                   endereco  \
0                                      <NA>   
1                                      <NA>   
2  SAIS Área Especial 3, Setor Policial Sul   
3                                      <NA>   
4                                      <NA>   

                                           descricao  \
0  Contratação de instituição para execução de se...   
1  Construção de Escola em Tempo Integral, Escola...   
2  Reajuste do Contrato 4

In [33]:
df_projetos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 24 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   id_unico                                800 non-null    string        
 1   nome                                    800 non-null    string        
 2   cep                                     362 non-null    string        
 3   endereco                                403 non-null    string        
 4   descricao                               800 non-null    string        
 5   funcao_social                           800 non-null    string        
 6   meta_global                             800 non-null    string        
 7   data_inicial_prevista                   798 non-null    datetime64[ns]
 8   data_final_prevista                     798 non-null    datetime64[ns]
 9   data_inicial_efetiva                    27 non-null   

In [34]:
df_executores.info()

<class 'pandas.core.frame.DataFrame'>
Index: 875 entries, 0 to 799
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id_unico  875 non-null    string 
 1   nome      872 non-null    string 
 2   codigo    872 non-null    float64
dtypes: float64(1), string(2)
memory usage: 27.3 KB


In [35]:
df_eixos.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 967 entries, 0 to 799
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id_unico   967 non-null    string 
 1   id         964 non-null    float64
 2   descricao  964 non-null    string 
dtypes: float64(1), string(2)
memory usage: 30.2 KB


In [36]:
df_tipos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 967 entries, 0 to 799
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id_unico   967 non-null    string 
 1   id         964 non-null    float64
 2   descricao  964 non-null    string 
 3   id_eixo    964 non-null    float64
dtypes: float64(2), string(2)
memory usage: 37.8 KB


In [37]:
df_subtipos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 967 entries, 0 to 799
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id_unico   967 non-null    string 
 1   id         964 non-null    float64
 2   descricao  964 non-null    string 
 3   id_tipo    964 non-null    float64
dtypes: float64(2), string(2)
memory usage: 37.8 KB


In [38]:
df_fontes_recurso.info()

<class 'pandas.core.frame.DataFrame'>
Index: 800 entries, 0 to 799
Data columns (total 3 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id_unico                     800 non-null    string 
 1   origem                       800 non-null    string 
 2   valor_investimento_previsto  800 non-null    float64
dtypes: float64(1), string(2)
memory usage: 25.0 KB


## 4) Limpeza adicional e remoção de colunas esparsas
Removeremos colunas com mais de 95% de valores nulos — são geralmente não informativas para análise inicial. Também vamos achatar colunas de listas quando possível (ex.: tomadores, executores) mantendo o comprimento máximo ou a primeira entrada.

In [39]:
# Remover colunas com frac de NA acima de um limiar
SPARSE_THRESHOLD = 0.95
na_frac = df.isna().mean()
sparse_cols = na_frac[na_frac > SPARSE_THRESHOLD].index.tolist()
print('Colunas muito esparsas (serao removidas):', len(sparse_cols))
# Exclui colunas esparsas
df = df.drop(columns=sparse_cols)

# Achatar colunas de listas para informacao resumida (se existirem)
list_cols = [c for c in df.columns if df[c].apply(lambda x: isinstance(x, list)).any()]
print('Colunas que contêm listas (serao resumidas):', list_cols)
for c in list_cols:
    # Converte lista vazia ou nao-lista para None, senao pega o primeiro elemento ou comprimento
    df[c + '_len'] = df[c].apply(lambda x: len(x) if isinstance(x, list) else (0 if pd.isna(x) else 1))
    # Também extrai o primeiro item se existir e for dict com 'nome' ou 'id'
    def first_summary(v):
        if isinstance(v, list) and len(v) > 0:
            first = v[0]
            if isinstance(first, dict):
                return first.get('nome') or first.get('id') or str(first)
            return first
        return pd.NA
    df[c + '_first'] = df[c].apply(first_summary)
    # opcional: drop original list column para evitar campos complexos
    df = df.drop(columns=[c])

print('Shape apos limpeza:', df.shape)
df.head(2)


Colunas muito esparsas (serao removidas): 2
Colunas que contêm listas (serao resumidas): ['tomadores', 'executores', 'repassadores', 'eixos', 'tipos', 'subTipos', 'fontesDeRecurso']
Shape apos limpeza: (800, 36)


,idUnico,nome,cep,endereco,descricao,funcaoSocial,metaGlobal,dataInicialPrevista,dataFinalPrevista,dataCadastro,...,repassadores_len,repassadores_first,eixos_len,eixos_first,tipos_len,tipos_first,subTipos_len,subTipos_first,fontesDeRecurso_len,fontesDeRecurso_first
0,50379.53-54,DL - 304/2024 - Contratação de instituição par...,<NA>,<NA>,Contratação de instituição para execução de se...,Ampliação da capacidade de trafego visando a m...,Projetos Básicos e Executivos de Engenharia,2024-12-20,2027-12-05,2024-12-20,...,0,<NA>,1,3,1,25,1,4,1,"{'origem': 'Federal', 'valorInvestimentoPrevis..."
1,42724.53-27,Escola Classe Crixá São Sebastião,<NA>,<NA>,"Construção de Escola em Tempo Integral, Escola...",A construção da nova escola beneficiará 977 es...,"Construção de Escola em Tempo Integral, Escola...",2024-09-02,2028-09-02,2024-08-30,...,1,FUNDO NACIONAL DE DESENVOLVIMENTO DA EDUCAÇÃO,1,4,1,46,1,84,1,"{'origem': 'Federal', 'valorInvestimentoPrevis..."


## 5) Salvando saída processada
Salvamos o resultado em `data/processed/` tanto em Parquet (compacto, preserva tipos) quanto em JSON legível.

In [40]:
df_projetos.to_parquet(
    str(PROCESSED_DIR) + '/dados_projetos_limpos.parquet', 
    engine='pyarrow',
    index=False
)


### Observações finais
- Ajustes adicionais (imputação, encoding categórico, feature engineering) devem ser feitos em notebooks posteriores com base nas necessidades do modelo/análise.
- Mantivemos um arquivo Parquet que preserva tipos e é eficiente para cargas futuras.